# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import joblib
import uuid
import pandas as pd 
import requests
import json
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.compute_target import ComputeTargetException
import os
from azureml.widgets import RunDetails
from azureml.core.runconfig import RunConfiguration, EnvironmentDefinition, CondaDependencies

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'automl-udacity'

experiment=Experiment(ws, experiment_name)

In [4]:
dataset = Dataset.get_by_name(ws, name='heart_data')

In [28]:
from azureml.core import Workspace, Dataset, Datastore

subscription_id = 'bfdbbde2-f093-45f7-9b9d-c1721eed4782'
resource_group = 't.tschuemperlin-rg'
workspace_name = 'final-project-udacity'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "workspaceblobstore")
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'UI/2023-11-13_084243_UTC/heart.csv'))
df = dataset.to_pandas_dataframe()
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "comp-udacity"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=16)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)
compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [6]:
# preliminaries
primary_metric_name = "accuracy"
if "training" not in os.listdir():
    os.mkdir("training/")

# setup training environment
task_env = Environment.from_pip_requirements(
    name="venv", 
    file_path="requirements.txt"
)

# policy specification
early_termination_policy = BanditPolicy(
    evaluation_interval=2, 
    slack_factor=0.2
)

# parameter sampler specification
param_sampling = RandomParameterSampling(
    {
        "--learning_rate": uniform(0.1, 0.5),
        "--n_estimators": choice(100, 200, 300, 350),
    }
)

# estimator and hyperdrive config specification
estimator = ScriptRunConfig(
    source_directory="./",
    script="train_validate.py",
    environment=task_env,
    compute_target=compute_target,
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4,
)

In [7]:
#TODO: Submit your experiment
hyper_drive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(hyper_drive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
hyper_drive_run.wait_for_completion(show_output=True)

RunId: HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703
Web View: https://ml.azure.com/runs/HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703?wsid=/subscriptions/bfdbbde2-f093-45f7-9b9d-c1721eed4782/resourcegroups/t.tschuemperlin-rg/workspaces/final-project-udacity&tid=7dba2f36-129a-4d4a-9ab4-1e18c9f46061

Execution Summary
RunId: HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703
Web View: https://ml.azure.com/runs/HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703?wsid=/subscriptions/bfdbbde2-f093-45f7-9b9d-c1721eed4782/resourcegroups/t.tschuemperlin-rg/workspaces/final-project-udacity&tid=7dba2f36-129a-4d4a-9ab4-1e18c9f46061



{'runId': 'HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703',
 'target': 'comp-udacity',
 'status': 'Completed',
 'startTimeUtc': '2023-11-19T12:18:42.584067Z',
 'endTimeUtc': '2023-11-19T12:22:15.101703Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e02eb83d-ea2d-4455-876d-97ca1d127383',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703_6',
  'score': '0.8421052631578947',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703_6'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryVal

In [12]:


hyper_drive_run.get_metrics()

RunId: HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703
Web View: https://ml.azure.com/runs/HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703?wsid=/subscriptions/bfdbbde2-f093-45f7-9b9d-c1721eed4782/resourcegroups/t.tschuemperlin-rg/workspaces/final-project-udacity&tid=7dba2f36-129a-4d4a-9ab4-1e18c9f46061

Execution Summary
RunId: HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703
Web View: https://ml.azure.com/runs/HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703?wsid=/subscriptions/bfdbbde2-f093-45f7-9b9d-c1721eed4782/resourcegroups/t.tschuemperlin-rg/workspaces/final-project-udacity&tid=7dba2f36-129a-4d4a-9ab4-1e18c9f46061



{'HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703_9': {'learning_rate:': 0.4315280488674128,
  'n_estimators:': 300,
  'clf_report': '              precision    recall  f1-score   support\n\n           0       0.89      0.69      0.77        35\n           1       0.78      0.93      0.84        41\n\n    accuracy                           0.82        76\n   macro avg       0.83      0.81      0.81        76\nweighted avg       0.83      0.82      0.81        76\n',
  'accuracy': 0.8157894736842105},
 'HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703_8': {'learning_rate:': 0.44074159672821567,
  'n_estimators:': 300,
  'clf_report': '              precision    recall  f1-score   support\n\n           0       0.89      0.69      0.77        35\n           1       0.78      0.93      0.84        41\n\n    accuracy                           0.82        76\n   macro avg       0.83      0.81      0.81        76\nweighted avg       0.83      0.82      0.81        76\n',
  'accuracy': 0.8157894736842105},
 'H

In [10]:
brun = hyper_drive_run.get_best_run_by_primary_metric()
brun_metrics = brun.get_metrics()
parameter_values = brun.get_details()["runDefinition"]["arguments"]
brun_acc = brun_metrics["accuracy"]
print(f"ID for best model run: {brun.id}\n")
print(f"Used hyperparameters for the best model: {parameter_values}\n")
print(f"Best metrics collected from the best run: {brun_metrics}\n")
print(f"Reached accuracy: {brun_metrics['accuracy']}\n")

ID for best model run: HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703_6

Used hyperparameters for the best model: ['--learning_rate', '0.292486235382985', '--n_estimators', '100']

Best metrics collected from the best run: {'learning_rate:': 0.292486235382985, 'n_estimators:': 100, 'clf_report': '              precision    recall  f1-score   support\n\n           0       0.79      0.89      0.84        35\n           1       0.89      0.80      0.85        41\n\n    accuracy                           0.84        76\n   macro avg       0.84      0.85      0.84        76\nweighted avg       0.85      0.84      0.84        76\n', 'accuracy': 0.8421052631578947}

Reached accuracy: 0.8421052631578947



## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [33]:
from azureml.core import Model, Run
from azureml.core.resource_configuration import ResourceConfiguration

# Registering the model
model_name = "hyperdrive_model"
model = brun.register_model(
    model_name= model_name,
    model_path="./outputs/model.joblib",
    resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1)
)

# Display model information
print(f"name: {model.name}")
print(f"version: {model.version}")
print(f"run id: {model.run_id}")

Model Name: hyperdrive_model
Version: 6
Run ID: HD_58e42407-cf3f-4bec-81e7-a70ebd9d3703_6


TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

task_env = Environment.from_pip_requirements(
    name="venv", 
    file_path="requirements.txt"
)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
)

inference_config = InferenceConfig(
    entry_script="score.py",
    environment=task_env,
)

service = Model.deploy(
    ws,
    "capstone-service",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-19 12:39:19+00:00 Creating Container Registry if not exists.
2023-11-19 12:39:19+00:00 Registering the environment.
2023-11-19 12:39:21+00:00 Use the existing image.
2023-11-19 12:39:22+00:00 Generating deployment configuration.
2023-11-19 12:39:22+00:00 Submitting deployment to compute.
2023-11-19 12:39:25+00:00 Checking the status of deployment capstone-service..
2023-11-19 12:42:39+00:00 Checking the status of inference endpoint capstone-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [17]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_58f440cd5db21b9ab3c363f087e7b355/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_58f440cd5db21b9ab3c363f087e7b355/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_58f440cd5db21b9ab3c363f087e7b355/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-11-19T12:41:10,504953719+00:00 - gunicorn/run 
2023-11-19T12:41:10,504915419+00:00 - rsyslog/run 
2023-11-19T12:41:10,518635766+00:00 | gunicorn/run | 
2023-11-19T12:41:10,521490675+00:00 | gunicorn/run | ###############################################
2023-11-19T12:41:10,529468302+00:00 | gunicorn/run | AzureML Container Runtime Information
bash: /azureml-envs/azureml_58f440cd5db21b9ab3c363f087e7b355/lib/libtinfo.so.6: no version information available (required by bash)
2023-11-19T12:41:10,539293536+00:00 | gunicorn/run | ########################################

In [34]:
target_var = "output"
df_data = df.drop(columns=target_var).sample(n=5).to_dict("records")
json_payload = {
    "data": df_data,
}

dataraw = json.dumps(json_payload)
display(df_data)

[{'age': 57,
  'sex': 1,
  'cp': 0,
  'trtbps': 110,
  'chol': 201,
  'fbs': 0,
  'restecg': 1,
  'thalachh': 126,
  'exng': 1,
  'oldpeak': 1.5,
  'slp': 1,
  'caa': 0,
  'thall': 1},
 {'age': 57,
  'sex': 1,
  'cp': 0,
  'trtbps': 130,
  'chol': 131,
  'fbs': 0,
  'restecg': 1,
  'thalachh': 115,
  'exng': 1,
  'oldpeak': 1.2,
  'slp': 1,
  'caa': 1,
  'thall': 3},
 {'age': 52,
  'sex': 1,
  'cp': 2,
  'trtbps': 138,
  'chol': 223,
  'fbs': 0,
  'restecg': 1,
  'thalachh': 169,
  'exng': 0,
  'oldpeak': 0.0,
  'slp': 2,
  'caa': 4,
  'thall': 2},
 {'age': 41,
  'sex': 0,
  'cp': 1,
  'trtbps': 126,
  'chol': 306,
  'fbs': 0,
  'restecg': 1,
  'thalachh': 163,
  'exng': 0,
  'oldpeak': 0.0,
  'slp': 2,
  'caa': 0,
  'thall': 2},
 {'age': 67,
  'sex': 1,
  'cp': 0,
  'trtbps': 125,
  'chol': 254,
  'fbs': 1,
  'restecg': 1,
  'thalachh': 163,
  'exng': 0,
  'oldpeak': 0.2,
  'slp': 1,
  'caa': 2,
  'thall': 3}]

In [35]:
reqheaders = {"Content-Type": "application/json"}
uri = service.scoring_uri

print(f"Service scoring URI: {uri}")
response = requests.post(uri, data=dataraw, headers=reqheaders)
print(response.json())

Service scoring URI: http://b30554a0-0685-4713-a9b1-781cb4b18fd3.eastus.azurecontainer.io/score
[1, 0, 1, 1, 0]


In [38]:
service.delete()
model.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

